For more details on this tutorial's setup and process, see `00_Introduction.ipynb`.

# Stage 3: Deploy the Model in Wallaroo
 
In this stage, we upload the trained model and the processing steps to Wallaroo, then set up and deploy the inference pipeline. 

Once deployed we can feed the newest batch of data to the pipeline, do the inferences and write the results to a results table.

For clarity in this demo, we have split the training/upload task into two notebooks:

* `02_automated_training_process.ipynb`: Train and pickle ML model.
* `03_deploy_model.ipynb`: Upload the model to Wallaroo and deploy into a pipeline.

Assuming no changes are made to the structure of the model, these two notebooks, or a script based on them, can then be scheduled to run on a regular basis, to refresh the model with more recent training data and update the inference pipeline.

This notebook is expected to run within the Wallaroo instance's Jupyter Hub service to provide access to all required Wallaroo libraries and functionality.

## Resources

The following resources are used as part of this tutorial:

* **data**
  * `data/seattle_housing_col_description.txt`: Describes the columns used as part data analysis.
  * `data/seattle_housing.csv`: Sample data of the Seattle, Washington housing market between 2014 and 2015.
* **code**
  * `postprocess.py`: Formats the data after inference by the model is complete.
  * `preprocess.py`: Formats the incoming data for the model.
  * `simdb.py`: A simulated database to demonstrate sending and receiving queries.
  * `wallaroo_client.py`: Additional methods used with the Wallaroo instance to create workspaces, etc.
* **models**
  * `housing_model_xgb.onnx`: Model created in Stage 2: Training Process Automation Setup.

## Steps

The process of uploading the model to Wallaroo follows these steps:

* [Connect to Wallaroo](#connect-to-wallaroo): Connect to the Wallaroo instance and set up the workspace.
* [Upload The Model](#upload-the-model): Upload the model and autoconvert for use in the Wallaroo engine.
* [Upload the Processing Modules](#upload-the-processing-modules): Upload the processing modules.
* [Create and Deploy the Pipeline](#create-and-deploy-the-pipeline): Create the pipeline with the model and processing modules as steps, then deploy it.
* [Test the Pipeline](#test-the-pipeline): Verify that the pipeline works with the sample data.

### Connect to Wallaroo

First we import the required libraries to connect to the Wallaroo instance, then connect to the Wallaroo instance.

In [1]:
import json
import pickle
import pandas as pd
import numpy as np

import simdb # module for the purpose of this demo to simulate pulling data from a database

from wallaroo.ModelConversion import ConvertXGBoostArgs, ModelConversionSource, ModelConversionInputType
import wallaroo
from wallaroo.object import EntityNotFoundError

In [2]:
wl = wallaroo.Client()

Please log into the following URL in a web browser:

	https://sparkly-apple-3026.keycloak.wallaroo.community/auth/realms/master/device?user_code=XRAD-UUUM

Login successful!


In [3]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [4]:
workspace_name = 'housepricing2'
model_name = "housepricemodel"
model_file = "./housing_model_xgb.onnx"
pipeline_name = "housing-pipe"

The workspace `housepricing` will either be created or, if already existing, used and set to the current workspace.

In [5]:
new_workspace = get_workspace(workspace_name)
new_workspace

{'name': 'housepricing2', 'id': 50, 'archived': False, 'created_by': '6e87ec2b-ad7f-4b0f-b426-5100c26944ba', 'created_at': '2022-10-18T19:41:52.672007+00:00', 'models': [{'name': 'housepricemodel', 'version': 'b66c8053-e28b-4f19-94ff-82f718e12681', 'file_name': 'housing_model_xgb.onnx', 'image_path': None, 'last_update_time': datetime.datetime(2022, 10, 18, 20, 30, 13, 695855, tzinfo=tzutc())}, {'name': 'preprocess', 'version': '09fda370-b5d6-42ef-93cf-429d3d116df3', 'file_name': 'preprocess.py', 'image_path': None, 'last_update_time': datetime.datetime(2022, 10, 18, 20, 30, 17, 364846, tzinfo=tzutc())}, {'name': 'postprocess', 'version': 'ac030fe2-4e86-4cdc-917e-4b5ad7b72838', 'file_name': 'postprocess.py', 'image_path': None, 'last_update_time': datetime.datetime(2022, 10, 18, 20, 30, 17, 766274, tzinfo=tzutc())}], 'pipelines': [{'name': 'housing-pipe', 'create_time': datetime.datetime(2022, 10, 18, 20, 30, 19, 448654, tzinfo=tzutc()), 'definition': '[]'}]}

In [6]:
_ = wl.set_current_workspace(new_workspace)

### Upload The Model

With the connection set and workspace prepared, upload the model created in `02_automated_training_process.ipynb` into the current workspace.

In [11]:
hpmodel = wl.upload_model(model_name, model_file).configure()

## Upload the Processing Modules

Upload the `preprocess.py` and `postprocess.py` modules as models to be added to the pipeline.

In [8]:
# load the preprocess module
module_pre = wl.upload_model("preprocess", "./preprocess.py").configure('python')

In [9]:
# load the postprocess module
module_post = wl.upload_model("postprocess", "./postprocess.py").configure('python')

### Create and Deploy the Pipeline

Create the pipeline with the preprocess module, housing model, and postprocess module as pipeline steps, then deploy the newpipeline.

In [12]:
pipeline = (wl.build_pipeline(pipeline_name)
              .add_model_step(module_pre)
              .add_model_step(hpmodel)
              .add_model_step(module_post)
              .deploy()
           )
pipeline

Waiting for deployment - this will take up to 45s ......... ok


name,housing-pipe
created,2022-10-18 20:30:19.448654+00:00
last_updated,2022-10-19 17:24:17.505573+00:00
deployed,True
tags,
steps,preprocess


### Test the Pipeline

We will use a single query from the simulated `housing_price` table and infer.  When successful, we will undeploy the pipeline to restore the resources back to the Kubernetes environment.

In [13]:
conn = simdb.simulate_db_connection()

# create the query
query = f"select * from {simdb.tablename} limit 1"
print(query)

# read in the data
singleton = pd.read_sql_query(query, conn)
conn.close()

singleton

select * from house_listings limit 1


,id,date,list_price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sale_price
0,7129300520,2022-03-07,221900.0,3,1.0,1180,5650,1.0,0,0,...,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,221900.0


In [14]:
result = pipeline.infer({'query': singleton.to_json()})
# just display the output
result[0].data()

Waiting for inference response - this will take up to 45s .. ok


[array([224852.])]

When finished, we undeploy the pipeline to return the resources back to the environment.

In [15]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ........................................... ok


name,housing-pipe
created,2022-10-18 20:30:19.448654+00:00
last_updated,2022-10-19 17:24:17.505573+00:00
deployed,False
tags,
steps,preprocess


With this stage complete, we can proceed to Stage 4: Regular Batch Inference.